# CIBC

## Advanced Analytics Case Competition 

### Submission By Team-1 

1. Bhavana Rao
2. Haritha Poornachandran
3. Vaibhav Batra
4. Yash Bhatnagar

_____________

# Part 2 - Stock Analysis and Web Scrapping

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import wrds
import re
import warnings

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
df_etf = pd.read_csv('Data/ETF_Data_final.csv')
df_cust = pd.read_csv('Data/Bank_Customers.csv')

## Data prep

In [3]:
#Different tickers
tickers = df_etf['Ticker'].unique()
df_tic = pd.DataFrame({'tickers':tickers})

len(tickers)

2321

In [4]:
#Industry Data mapped with correct codes
df2 = pd.read_csv('etf_industry_correct.csv')

df2.head()

,tickers,as_of_date,composite_ticker,issuer,description,primary_benchmark,tax_classification,aum,asset_class,category,development_class,distributor,administrator,portfolio_manager,transfer_agent,listing_exchange,lead_market_maker,rn,Listing exchange,Code
0,AAA,2021-12-31,AAA,Alternative Access Funds,AAF First Priority CLO Bond ETF,NaN,Regulated Investment Company,1.000800e+07,Fixed Income,Corporate,Developed Markets,"Quasar Distributors, LLC","U.S. Bancorp Fund Services, LLC",Peter Coppa,"U.S. Bancorp Fund Services, LLC","NYSE Arca, Inc.",Flow Traders,1,"NYSE Arca, Inc.",arcx
1,AAAU,2021-12-31,AAAU,Goldman Sachs,Goldman Sachs Physical Gold ETF,NaN,Grantor Trust,4.242234e+08,Commodities,Precious Metals,Blended Development,"ALPS Distributors, Inc","Goldman Sachs Asset Management, L.P.","Goldman Sachs Asset Management, L.P.","JPMorgan Chase Bank, N.A.","NYSE Arca, Inc.",Virtu Financial,1,"NYSE Arca, Inc.",bats
2,ABEQ,2021-12-31,ABEQ,Innovator Capital Management,Absolute Core Strategy ETF,NaN,Regulated Investment Company,5.545800e+07,Equity,Broad Equity,Developed Markets,"Northern Lights Distributors, LLC","Ultimus Fund Solutions, LLC",Robert J. Mark,Brown Brothers Harriman & Co.,"NYSE Arca, Inc.",GTS,1,"NYSE Arca, Inc.",arcx
3,ACES,2021-12-31,ACES,Alps,ALPS Clean Energy ETF,CIBC Atlas Clean Energy Index,Regulated Investment Company,8.747935e+08,Equity,Strategy,Developed Markets,"ALPS Portfolio Solutions Distributor, Inc.","ALPS Fund Services, Inc.","Ryan Mischker, Andrew Hicks",State Street Bank and Trust Company,"NYSE Arca, Inc.",GTS,1,"NYSE Arca, Inc.",arcx
4,ACTV,2021-12-31,ACTV,Redwood Investment Management,LeaderShares Activist Leaders ETF,NaN,Regulated Investment Company,8.101225e+07,Equity,Strategy,Developed Markets,"Northern Lights Distributors, LLC","Gemini Fund Services, LLC","Richard M. Duff, Michael T. Cheung",Brown Brothers Harriman & Co.,"NYSE Arca, Inc.",Virtu Financial,1,"NYSE Arca, Inc.",arcx


### Scrapped dataset -- > df_esgf

In [5]:
# Scapped data
df_esgf = pd.read_csv('ESG_data.csv')

df_esgf.head()

,Ticker,Environmental,Social,Governance,Sustainable Investment
0,AAA,NaN,NaN,NaN,No
1,AAAU,NaN,NaN,NaN,No
2,AADR,4.07,7.30,5.99,No
3,AAXJ,5.42,9.50,8.80,No
4,ABEQ,5.78,8.58,7.33,No


In [6]:
"""" 
Out of 2320 tickers
18 can't be scrapped
and 442 have null values in E, S and G columns
NOTE: They do have Sustainable Investment flag associated with them

"""
df_esgf.isna().sum()

Ticker                      0
Environmental             442
Social                    442
Governance                442
Sustainable Investment      0
dtype: int64

In [7]:
#Note: There are 18 tickers that can't be scapped from the net
print("Scrapped tickers length = ",len(df_esgf['Ticker'].unique()))
print("Industry tickers length = ",len(df2['tickers'].unique()))

Scrapped tickers length =  2302
Industry tickers length =  2320


In [8]:
#Tickers
scrapped_tickers = list(df_esgf['Ticker'].unique())
ind_tickers = list(df2['tickers'].unique())
ind_tickers_not = ind_tickers.copy()

#Finding which tickers are not present
for st in scrapped_tickers:
    if st in ind_tickers_not:
        ind_tickers_not.remove(st)

print("Total tickers = ",len(ind_tickers))
print("Not found tickers = ",len(ind_tickers_not))

Total tickers =  2320
Not found tickers =  18


In [9]:
#Removing not found tickers from the dataset
df3 = df2.copy()

df3.set_index('tickers',inplace=True)

for tc in ind_tickers_not:
    df3.drop(tc,inplace=True)

### Making df3 -- GOOD dataset

In [10]:
df3.reset_index(inplace=True)
df3.info() #Use df3 for further scrapping

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302 entries, 0 to 2301
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tickers             2302 non-null   object 
 1   as_of_date          2302 non-null   object 
 2   composite_ticker    2302 non-null   object 
 3   issuer              2302 non-null   object 
 4   description         2302 non-null   object 
 5   primary_benchmark   1926 non-null   object 
 6   tax_classification  2302 non-null   object 
 7   aum                 1968 non-null   float64
 8   asset_class         2302 non-null   object 
 9   category            2302 non-null   object 
 10  development_class   2302 non-null   object 
 11  distributor         2248 non-null   object 
 12  administrator       2244 non-null   object 
 13  portfolio_manager   2232 non-null   object 
 14  transfer_agent      2248 non-null   object 
 15  listing_exchange    2302 non-null   object 
 16  lead_m

In [11]:
df3.head()

,tickers,as_of_date,composite_ticker,issuer,description,primary_benchmark,tax_classification,aum,asset_class,category,development_class,distributor,administrator,portfolio_manager,transfer_agent,listing_exchange,lead_market_maker,rn,Listing exchange,Code
0,AAA,2021-12-31,AAA,Alternative Access Funds,AAF First Priority CLO Bond ETF,NaN,Regulated Investment Company,1.000800e+07,Fixed Income,Corporate,Developed Markets,"Quasar Distributors, LLC","U.S. Bancorp Fund Services, LLC",Peter Coppa,"U.S. Bancorp Fund Services, LLC","NYSE Arca, Inc.",Flow Traders,1,"NYSE Arca, Inc.",arcx
1,AAAU,2021-12-31,AAAU,Goldman Sachs,Goldman Sachs Physical Gold ETF,NaN,Grantor Trust,4.242234e+08,Commodities,Precious Metals,Blended Development,"ALPS Distributors, Inc","Goldman Sachs Asset Management, L.P.","Goldman Sachs Asset Management, L.P.","JPMorgan Chase Bank, N.A.","NYSE Arca, Inc.",Virtu Financial,1,"NYSE Arca, Inc.",bats
2,ABEQ,2021-12-31,ABEQ,Innovator Capital Management,Absolute Core Strategy ETF,NaN,Regulated Investment Company,5.545800e+07,Equity,Broad Equity,Developed Markets,"Northern Lights Distributors, LLC","Ultimus Fund Solutions, LLC",Robert J. Mark,Brown Brothers Harriman & Co.,"NYSE Arca, Inc.",GTS,1,"NYSE Arca, Inc.",arcx
3,ACES,2021-12-31,ACES,Alps,ALPS Clean Energy ETF,CIBC Atlas Clean Energy Index,Regulated Investment Company,8.747935e+08,Equity,Strategy,Developed Markets,"ALPS Portfolio Solutions Distributor, Inc.","ALPS Fund Services, Inc.","Ryan Mischker, Andrew Hicks",State Street Bank and Trust Company,"NYSE Arca, Inc.",GTS,1,"NYSE Arca, Inc.",arcx
4,ACTV,2021-12-31,ACTV,Redwood Investment Management,LeaderShares Activist Leaders ETF,NaN,Regulated Investment Company,8.101225e+07,Equity,Strategy,Developed Markets,"Northern Lights Distributors, LLC","Gemini Fund Services, LLC","Richard M. Duff, Michael T. Cheung",Brown Brothers Harriman & Co.,"NYSE Arca, Inc.",Virtu Financial,1,"NYSE Arca, Inc.",arcx


In [14]:
df3.columns

Index(['tickers', 'as_of_date', 'composite_ticker', 'issuer', 'description',
       'primary_benchmark', 'tax_classification', 'aum', 'asset_class',
       'category', 'development_class', 'distributor', 'administrator',
       'portfolio_manager', 'transfer_agent', 'listing_exchange',
       'lead_market_maker', 'rn', 'Listing exchange', 'Code'],
      dtype='object')

In [15]:
df4 = df3[['tickers','aum','asset_class','category','development_class']].copy()

In [16]:
df_web_run = pd.read_csv('returns_data.csv')
df_web_run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302 entries, 0 to 2301
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ticker  2302 non-null   object
 1   1year   2302 non-null   object
 2   3year   2302 non-null   object
 3   5year   2302 non-null   object
dtypes: object(4)
memory usage: 72.1+ KB


In [ ]:
df4,df_esgf,df_web_run,df

In [29]:
df_etf["rank"] = df_etf.groupby('Ticker')["month"].rank(ascending=False)


In [32]:
df_etf_latest = df_etf[df_etf['rank']==1]

In [18]:
df_etf.columns

Index(['Name', 'Ticker', 'Base Currency', 'Global Broad Category Group',
       'Global Category', 'Morningstar Category', 'Investment Area',
       'Fund Size Base Currency', 'Domicile', 'month',
       'Monthly Return  Base Currency',
       'Total Market Value(Net)  Portfolio Currency', 'asset_class',
       'Morningstar Rating Overall', 'Carbon Risk Classification',
       'Carbon Risk Score',
       'Corporate Sustainability Percent Rank in Global Category',
       'Fossil Fuel Involvement', 'Low Carbon Designation',
       'Morningstar Sustainability Rating', 'Oil & Gas Generation Involvement',
       'Percent of AUM Covered - Carbon',
       'Percent of AUM with High & Severe Controversies',
       'Percent of AUM with High Carbon Risk',
       'Percent of AUM with Low Carbon Risk',
       'Percent of AUM with Medium Carbon Risk',
       'Percent of AUM with Severe Carbon Risk',
       'Percent of AUM with Significant Controversies',
       'Portfolio Environmental Risk Score', 

In [34]:
df_e4 = df_etf_latest[['Name', 'Ticker','Morningstar Category','Investment Area','asset_class',
                       'Morningstar Rating Overall','Carbon Risk Classification',
                       'Fossil Fuel Involvement','Morningstar Sustainability Rating',
                       'Portfolio Environmental Risk Score', 'Portfolio Governance Risk Score',
       'Portfolio Social Risk Score',
       'Sustainable Investment - ESG Fund Overall',
       'Sustainable Investment - Environmental',
       'Sustainable Investment - Gender & Diversity',
       'Sustainable Investment - Low Carbon/Fossil-Fuel Free',
       'Sustainable Investment - Overall',
       'Sustainable Investment - Renewable Energy'                       
               ]].copy()

In [36]:
df_e4.isna().sum()

Name                                                       0
Ticker                                                     0
Morningstar Category                                       1
Investment Area                                         1791
asset_class                                                0
Morningstar Rating Overall                               882
Carbon Risk Classification                               761
Fossil Fuel Involvement                                  146
Morningstar Sustainability Rating                        302
Portfolio Environmental Risk Score                       352
Portfolio Governance Risk Score                          352
Portfolio Social Risk Score                              352
Sustainable Investment - ESG Fund Overall                 31
Sustainable Investment - Environmental                    31
Sustainable Investment - Gender & Diversity               31
Sustainable Investment - Low Carbon/Fossil-Fuel Free      31
Sustainable Investment -

___________

## WRDS - Analytics data

In [37]:
df_analytics = pd.read_csv('etf_analytics_good.csv')

In [38]:
df_analytics.head()

,tickers,risk_total_score,reward_score,risk_efficiency,risk_liquidity,risk_volatility,quant_total_score,quant_technical_shortterm,quant_technical_intermediate,quant_technical_long,quant_sentiment_shortinterest,quant_compositebehavioral,quant_fundamental_p_e,quant_fundamental_div,quant_global_sector,quant_quality_liquidity,quant_quality_diversification,quant_grade,quant_quality_firm
0,AAAU,4.07,0.00,3.24,5.00,3.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AADR,7.30,7.69,6.76,6.81,4.62,65.3,78.6,73.4,64.6,7.2,53.2995,67.5,69.5,25.5,13.8,82.5,B,68.7
2,AAXJ,5.54,4.34,5.86,2.84,4.84,48.3,55.4,41.6,56.9,81.7,49.8157,2.4,100.0,19.5,93.2,30.5,C,99.0
3,ACES,7.11,5.41,6.02,4.94,5.35,55.3,52.3,53.3,40.9,50.0,55.2774,50.0,53.5,50.6,51.2,56.8,B,78.7
4,ACSI,6.30,5.71,6.06,5.95,3.37,53.4,64.1,75.2,77.7,50.0,61.1397,50.0,46.5,22.4,31.0,70.4,C,15.7


In [41]:
df_analytics.columns

Index(['tickers', 'risk_total_score', 'reward_score', 'risk_efficiency',
       'risk_liquidity', 'risk_volatility', 'quant_total_score',
       'quant_technical_shortterm', 'quant_technical_intermediate',
       'quant_technical_long', 'quant_sentiment_shortinterest',
       'quant_compositebehavioral', 'quant_fundamental_p_e',
       'quant_fundamental_div', 'quant_global_sector',
       'quant_quality_liquidity', 'quant_quality_diversification',
       'quant_grade', 'quant_quality_firm'],
      dtype='object')

In [42]:
lst = ['tickers', 'risk_total_score', 'reward_score','quant_fundamental_p_e',
       'quant_fundamental_div','quant_grade', 'quant_quality_firm','quant_quality_diversification',
    'quant_global_sector']

df_anal = df_analytics[lst].copy()

In [43]:
df_anal

,tickers,risk_total_score,reward_score,quant_fundamental_p_e,quant_fundamental_div,quant_grade,quant_quality_firm,quant_quality_diversification,quant_global_sector
0,AAAU,4.07,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,AADR,7.30,7.69,67.5,69.5,B,68.7,82.5,25.5
2,AAXJ,5.54,4.34,2.4,100.0,C,99.0,30.5,19.5
3,ACES,7.11,5.41,50.0,53.5,B,78.7,56.8,50.6
4,ACSI,6.30,5.71,50.0,46.5,C,15.7,70.4,22.4
...,...,...,...,...,...,...,...,...,...
1598,YANG,8.65,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1599,YINN,9.34,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1600,YLDE,5.94,6.55,50.0,65.0,B,47.7,78.3,32.2
1601,YXI,7.50,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_a1 = pd.merge(left = df4, right = df_e4, left_on = 'tickers' , right_on='Ticker', how='left')
df_a2 = pd.merge(left = df_a1, right = df_web_run, left_on = 'tickers', right_on = 'Ticker', how='left')
df_a3 = pd.merge(left = df_a2, right = df_anal, left_on = 'tickers', right_on = 'tickers', how = 'left')
df_a3

,tickers,aum,asset_class_x,category,development_class,Name,Ticker_x,Morningstar Category,Investment Area,asset_class_y,...,3year,5year,risk_total_score,reward_score,quant_fundamental_p_e,quant_fundamental_div,quant_grade,quant_quality_firm,quant_quality_diversification,quant_global_sector
0,AAA,1.000800e+07,Fixed Income,Corporate,Developed Markets,AAF First Priority CLO Bond ETF,AAA,US Fund Ultrashort Bond,NaN,bond,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAAU,4.242234e+08,Commodities,Precious Metals,Blended Development,Goldman Sachs Physical Gold ETF,AAAU,US Fund Commodities Focused,NaN,commodity,...,13.72,-,4.07,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,ABEQ,5.545800e+07,Equity,Broad Equity,Developed Markets,Absolute Core Strategy ETF,ABEQ,US Fund Large Value,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACES,8.747935e+08,Equity,Strategy,Developed Markets,ALPS Clean Energy ETF,ACES,US Fund Miscellaneous Sector,NaN,equity,...,33.89,-,7.11,5.41,50.0,53.5,B,78.7,56.8,50.6
4,ACTV,8.101225e+07,Equity,Strategy,Developed Markets,LeaderShares® Activist Leaders® ETF,ACTV,US Fund Small Value,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,QQC,NaN,Equity,Size and Style,Developed Markets,NaN,NaN,NaN,NaN,NaN,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2298,QQD,NaN,Equity,Size and Style,Developed Markets,Simplify Nasdaq 100 plus Dwnsd cnvxtyETF,QQD,US Fund Large Growth,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2299,SKYU,NaN,Equity,Strategy,Blended Development,ProShares Ultra Cloud Computing,SKYU,US Fund Trading--Inverse Equity,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,VPN,8.078967e+07,Equity,Strategy,Blended Development,NaN,NaN,NaN,NaN,NaN,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_a3.isna().sum()

tickers                                                    0
aum                                                      334
asset_class_x                                              0
category                                                   0
development_class                                          0
Name                                                     269
Ticker_x                                                 269
Morningstar Category                                     269
Investment Area                                         2044
asset_class_y                                            269
Morningstar Rating Overall                              1138
Carbon Risk Classification                              1019
Fossil Fuel Involvement                                  407
Morningstar Sustainability Rating                        564
Portfolio Environmental Risk Score                       614
Portfolio Governance Risk Score                          614
Portfolio Social Risk Sc

In [50]:
df_final = df_a3.copy()

In [51]:
df_final.head()

,tickers,aum,asset_class_x,category,development_class,Name,Ticker_x,Morningstar Category,Investment Area,asset_class_y,...,3year,5year,risk_total_score,reward_score,quant_fundamental_p_e,quant_fundamental_div,quant_grade,quant_quality_firm,quant_quality_diversification,quant_global_sector
0,AAA,1.000800e+07,Fixed Income,Corporate,Developed Markets,AAF First Priority CLO Bond ETF,AAA,US Fund Ultrashort Bond,NaN,bond,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAAU,4.242234e+08,Commodities,Precious Metals,Blended Development,Goldman Sachs Physical Gold ETF,AAAU,US Fund Commodities Focused,NaN,commodity,...,13.72,-,4.07,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,ABEQ,5.545800e+07,Equity,Broad Equity,Developed Markets,Absolute Core Strategy ETF,ABEQ,US Fund Large Value,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACES,8.747935e+08,Equity,Strategy,Developed Markets,ALPS Clean Energy ETF,ACES,US Fund Miscellaneous Sector,NaN,equity,...,33.89,-,7.11,5.41,50.0,53.5,B,78.7,56.8,50.6
4,ACTV,8.101225e+07,Equity,Strategy,Developed Markets,LeaderShares® Activist Leaders® ETF,ACTV,US Fund Small Value,NaN,equity,...,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_esgf

,Ticker,Environmental,Social,Governance,Sustainable Investment
0,AAA,NaN,NaN,NaN,No
1,AAAU,NaN,NaN,NaN,No
2,AADR,4.07,7.30,5.99,No
3,AAXJ,5.42,9.50,8.80,No
4,ABEQ,5.78,8.58,7.33,No
...,...,...,...,...,...
2297,YSEP,NaN,NaN,NaN,No
2298,YXI,NaN,NaN,NaN,No
2299,ZECP,3.55,9.43,7.01,No
2300,ZIG,2.34,8.28,6.73,No


In [54]:
df_final = pd.merge(left = df_a3, right = df_esgf, left_on='tickers',right_on='Ticker',how='left')
df_final

,tickers,aum,asset_class_x,category,development_class,Name,Ticker_x,Morningstar Category,Investment Area,asset_class_y,...,quant_fundamental_div,quant_grade,quant_quality_firm,quant_quality_diversification,quant_global_sector,Ticker,Environmental,Social,Governance,Sustainable Investment
0,AAA,1.000800e+07,Fixed Income,Corporate,Developed Markets,AAF First Priority CLO Bond ETF,AAA,US Fund Ultrashort Bond,NaN,bond,...,NaN,NaN,NaN,NaN,NaN,AAA,NaN,NaN,NaN,No
1,AAAU,4.242234e+08,Commodities,Precious Metals,Blended Development,Goldman Sachs Physical Gold ETF,AAAU,US Fund Commodities Focused,NaN,commodity,...,NaN,NaN,NaN,NaN,NaN,AAAU,NaN,NaN,NaN,No
2,ABEQ,5.545800e+07,Equity,Broad Equity,Developed Markets,Absolute Core Strategy ETF,ABEQ,US Fund Large Value,NaN,equity,...,NaN,NaN,NaN,NaN,NaN,ABEQ,5.78,8.58,7.33,No
3,ACES,8.747935e+08,Equity,Strategy,Developed Markets,ALPS Clean Energy ETF,ACES,US Fund Miscellaneous Sector,NaN,equity,...,53.5,B,78.7,56.8,50.6,ACES,4.30,6.33,4.29,Yes
4,ACTV,8.101225e+07,Equity,Strategy,Developed Markets,LeaderShares® Activist Leaders® ETF,ACTV,US Fund Small Value,NaN,equity,...,NaN,NaN,NaN,NaN,NaN,ACTV,4.96,6.62,4.21,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,QQC,NaN,Equity,Size and Style,Developed Markets,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,QQC,2.89,9.87,7.56,No
2298,QQD,NaN,Equity,Size and Style,Developed Markets,Simplify Nasdaq 100 plus Dwnsd cnvxtyETF,QQD,US Fund Large Growth,NaN,equity,...,NaN,NaN,NaN,NaN,NaN,QQD,2.89,9.87,7.56,No
2299,SKYU,NaN,Equity,Strategy,Blended Development,ProShares Ultra Cloud Computing,SKYU,US Fund Trading--Inverse Equity,NaN,equity,...,NaN,NaN,NaN,NaN,NaN,SKYU,1.37,8.15,5.98,No
2300,VPN,8.078967e+07,Equity,Strategy,Blended Development,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VPN,4.33,5.27,5.86,No


In [55]:
df_final.to_csv('Final_merged2.csv',index=False,header=True)

________

## Web Scapping 

### Source: https://www.morningstar.com/ 

### Part A : Scraping ESG scores and sustainability of a particular stock

_____________________


### Part B : Scraping returns of a particular stock

## Thank You!

By Team-1

__________